'''
@Author: Naziya

@Date: 2021-09-16

@Last Modified by: Naziya

@Last Modified : 2021-09-16

@Title : Program Aim is to create a dataframe using spark and add it into the hive table.

'''

Creating spark sesion with hive support enable

In [1]:

from pyspark.sql import SparkSession
from LoggerFormat import logger
try:
    appName = "PySpark MySQL Example - via mysql.connector"
    master = "local"
    spark = SparkSession.builder.master(master).appName(appName).enableHiveSupport().getOrCreate()
except Exception as e:
    logger.error(e)


21/09/16 18:48:20 WARN Utils: Your hostname, naziya-Inspiron-N5110 resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface wlp9s0)
21/09/16 18:48:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/16 18:48:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).



Creating dataframe of csv file using spark and selecting only certain columns


In [9]:
try:
    df = spark.read.csv("hdfs://localhost:9000/spark_sql/merged_CpuLogData.csv",header=True)
    df2 = df.select("user_name","DateTime","keyboard","mouse")
    df2.createOrReplaceTempView('myview')
except Exception as e:
    logger.error(e)

21/09/16 18:50:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
df2.show()

+--------------------+-------------------+--------+------+
|           user_name|           DateTime|keyboard| mouse|
+--------------------+-------------------+--------+------+
|rahilstar11@gmail...|2019-09-16 12:55:03|     0.0|   0.0|
|salinabodale73@gm...|2019-09-16 12:55:02|  2919.5| 888.0|
|bhagyashrichalke2...|2019-09-16 12:55:01|   144.0|2886.0|
|bhagyashrichalke2...|2019-09-16 13:00:01|    21.0|  44.0|
|  iamnzm@outlook.com|2019-09-16 13:00:01|    41.0|8251.0|
|deepshukla292@gma...|2019-09-16 13:00:01|   249.5|4266.0|
|salinabodale73@gm...|2019-09-16 13:00:02|   135.5| 692.0|
|sharlawar77@gmail...|2019-09-16 13:00:04|   303.0| 243.0|
|rahilstar11@gmail...|2019-09-16 13:00:03|    22.5| 170.0|
|salinabodale73@gm...|2019-09-16 13:05:01|   179.5| 108.0|
|bhagyashrichalke2...|2019-09-16 13:05:02|   387.0| 676.0|
|deepshukla292@gma...|2019-09-16 13:05:01|   165.0|5701.0|
|  iamnzm@outlook.com|2019-09-16 13:05:02|    52.0|7971.0|
|sharlawar77@gmail...|2019-09-16 13:05:04|    17.0|  72.

In [11]:
df10 = spark.sql("show databases")
df10.show()

+---------+
|namespace|
+---------+
|  default|
|     test|
+---------+




Now Adding dataframe into hive table using different methods


Method 1

In [12]:
try:
    # Save df to a new table in Hive we just need database table name we can give anything
    df2.write.mode("overwrite").saveAsTable("test.test_data")

except Exception as e:
    logger.error(e)

21/09/16 18:50:59 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.metastore.wm.default.pool.size does not exist
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.llap.task.scheduler.preempt.independent does not exist
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.llap.output.format.arrow does not exist
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.tez.llap.min.reducer.per.executor does not exist
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.arrow.root.allocator.limit does not exist
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.vectorized.use.checked.expressions does not exist
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.tez.dynamic.semijoin.reduction.for.mapjoin does not exist
21/09/16 18:51:00 WARN HiveConf: HiveConf of name hive.vectorized.complex.types.enabled does not exist
21/09/16 


Method 2


Loading spark dataframe by creating view and inserting it into hive


In [13]:
try:
    # here test is a database name and just give anything for table name it will create automatically
    spark.sql("create table test.test_data2 as select * from myview")

except Exception as e:
    logger.error(e)

21/09/16 18:51:16 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
21/09/16 18:51:17 WARN HiveMetaStore: Location: hdfs://localhost:9000/user/hive/warehouse/test.db/test_data2 specified for non-external table:test_data2


Showing The hive tables

In [14]:
try:
    
    dfs = spark.sql("use test")
    dfs1 = spark.sql("show tables")
    dfs1.show()
    

except Exception as e:
    logger.error(e)

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|    test| test_data|      false|
|    test|test_data2|      false|
|        |    myview|       true|
+--------+----------+-----------+

